## 05/03/19

- Extracting S1/S2 segments from raw files.

In [13]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import parser
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks, get_s1s2
from IPython.display import display, clear_output, HTML

import pdb

plt.style.use('default')

file_loc = '/Users/matthewashman/github/MasterProject2018/Data/EPdata/'
with open('/Users/matthewashman/github/MasterProject2018/non_af_labels.csv', 'r') as file_in:
    labels = pd.read_csv(file_in)

In [10]:
af_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10') # AF patients
at_patients = ('1', '2', '3') # AT patients
avnrt_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22') # AVNRT patients
avrt_patients = ('1', '2', '3', '4', '5', '6', '7', '8') # AVRT patients
ep_patients = ('1', '2', '3', '4', '5', '6', '7', '8') # EP patients
coupling_intervals = ('220','230', '240', '250', '260', '270', '280', '290', '300',
                      '310', '320', '330', '340', '350', '360', '370', '380', '390', '400')

X_af = epdata_main(file_loc, af_patients, 'af', tuple(reversed(coupling_intervals)))
X_at = epdata_main(file_loc, at_patients, 'at', list(reversed(coupling_intervals)))
X_avnrt = epdata_main(file_loc, avnrt_patients, 'avnrt', list(reversed(coupling_intervals)))
X_anrt = epdata_main(file_loc, avrt_patients, 'avrt', list(reversed(coupling_intervals)))
X_ep = epdata_main(file_loc, ep_patients, 'ep', list(reversed(coupling_intervals)))

X = pd.concat([X_af, X_at, X_avnrt, X_anrt, X_ep], ignore_index=True)

Parsing error: coupling interval 400ms not available for patient 1.
Parsing error: coupling interval 390ms not available for patient 1.
Parsing error: coupling interval 380ms not available for patient 1.
Parsing error: coupling interval 370ms not available for patient 1.
Parsing error: coupling interval 360ms not available for patient 1.
Parsing error: coupling interval 350ms not available for patient 1.
Parsing error: coupling interval 220ms not available for patient 1.
Parsing error: coupling interval 240ms not available for patient 2.
Parsing error: coupling interval 230ms not available for patient 2.
Parsing error: coupling interval 220ms not available for patient 2.
Parsing error: coupling interval 250ms not available for patient 3.
Parsing error: coupling interval 240ms not available for patient 3.
Parsing error: coupling interval 230ms not available for patient 3.
Parsing error: coupling interval 220ms not available for patient 3.
Parsing error: coupling interval 400ms not avail

Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 280ms.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 270ms.
Parsing error: coupling interval 260ms not available for patient 5.
Parsing error: coupling interval 250ms not available for patient 5.
Parsing error: coupling interval 240ms not available for patient 5.
Parsing error: coupling interval 230ms not available for patient 5.
Parsing error: coupling interval 220ms not available for patient 5.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 400ms.
Parsing error: coupling interval 390ms not available for patient 6.
Parsing error: coupling interval 380ms not available for patient 6.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 370ms.
Parsing error: coupling interval 360ms not available for patient 6.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 350ms.
Parsing error: non-integer valu

Parsing error: coupling interval 310ms not available for patient 17.
Parsing error: coupling interval 270ms not available for patient 17.
Parsing error: coupling interval 260ms not available for patient 17.
Parsing error: coupling interval 250ms not available for patient 17.
Parsing error: coupling interval 240ms not available for patient 17.
Parsing error: coupling interval 230ms not available for patient 17.
Parsing error: coupling interval 220ms not available for patient 17.
Parsing error: coupling interval 400ms not available for patient 18.
Parsing error: coupling interval 350ms not available for patient 18.
Parsing error: coupling interval 330ms not available for patient 18.
Parsing error: coupling interval 310ms not available for patient 18.
Parsing error: coupling interval 240ms not available for patient 18.
Parsing error: coupling interval 230ms not available for patient 18.
Parsing error: coupling interval 220ms not available for patient 18.
Parsing error: coupling interval 2

Parsing error: coupling interval 240ms not available for patient 7.
Parsing error: coupling interval 230ms not available for patient 7.
Parsing error: coupling interval 220ms not available for patient 7.
Parsing error: coupling interval 270ms not available for patient 8.
Parsing error: coupling interval 260ms not available for patient 8.
Parsing error: coupling interval 250ms not available for patient 8.
Parsing error: coupling interval 240ms not available for patient 8.
Parsing error: coupling interval 230ms not available for patient 8.
Parsing error: coupling interval 220ms not available for patient 8.
Parsing error: coupling interval 390ms not available for patient 1.
Parsing error: coupling interval 370ms not available for patient 1.
Parsing error: coupling interval 360ms not available for patient 1.
Parsing error: coupling interval 350ms not available for patient 1.
Parsing error: coupling interval 330ms not available for patient 1.
Parsing error: coupling interval 310ms not avail

In [14]:
labels = labels.dropna(how='all')
X['Label 1'] = np.nan; X['Label 2'] = np.nan
for idx, row in labels.iterrows():
    patient = row['Patient']
    coupling_interval = str(int(row['Coupling Interval']))
    patient_num = ''.join(filter(str.isdigit, patient))
    patient_type = ''.join(filter(str.isalpha, patient))
    patient_type = patient_type.lower()
    # Add 0/1 labels to dataframe
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS1-2') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 1'] = row['CS1-2']
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS3-4') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 1'] = row['CS3-4']
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS5-6') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 1'] = row['CS5-6']
    # Add 0/1/2 labels to dataframe
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS1-2') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 2'] = row["CS1-2'"]
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS3-4') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 2'] = row["CS3-4'"]
    X.loc[(X['Type']==patient_type) & (X['Patient']==patient_num) & (X['Channel']=='CS5-6') & (X['Coupling Interval']==coupling_interval) & (X['S1/S2']=='S2'), 'Label 2'] = row["CS5-6'"]

In [15]:
X[(X['Type']=='af') & (X['Patient']=='7')]

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label 1,Label 2
687,CS1-2,400,"[367, 274, 216, 217, 218, 237, 243, 234, 138, ...",7,S2,af,-1,-1
688,CS1-2,400,"[264.0, 249.0, 215.0, 195.0, 144.0, 144.0, 186...",7,S1,af,NaN,NaN
689,CS1-2,400,"[224.0, 215.0, 192.0, 194.0, 165.0, 100.0, 148...",7,S1,af,NaN,NaN
690,CS3-4,400,"[588, 500, 495, 528, 692, 752, 864, 795, 700, ...",7,S2,af,0,0
691,CS3-4,400,"[320.0, 187.0, -49.0, -23.0, 13.0, 10.0, -2.0,...",7,S1,af,NaN,NaN
692,CS3-4,400,"[261.0, 25.0, 3.0, 9.0, -44.0, -31.0, -44.0, -...",7,S1,af,NaN,NaN
693,CS5-6,400,"[924, 461, 221, 253, 386, 405, 504, 553, 477, ...",7,S2,af,-1,-1
694,CS5-6,400,"[960.0, 634.0, 94.0, 53.0, 98.0, 102.0, -171.0...",7,S1,af,NaN,NaN
695,CS5-6,400,"[792.0, 233.0, 65.0, 81.0, 61.0, -65.0, -395.0...",7,S1,af,NaN,NaN
696,CS1-2,380,"[155, 178, 179, 187, 259, 244, 224, 295, 253, ...",7,S2,af,-1,-1


In [16]:
X.to_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_channel_labels_new.pkl')

In [8]:
%matplotlib qt
patient_types = X['Type'].unique()
M=14
v = np.ones(M)

patient_types = ('af', 'avrt')

for patient_type in patient_types:
    patients = X[X['Type']==patient_type]['Patient'].unique()
    if patient_type == 'avrt':
        patients = patients[1:]
        
    for patient in patients:
        patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='S2')]
        
        patient_cis = patient_X['Coupling Interval'].unique()

        fig, axes = plt.subplots(nrows=len(patient_cis), ncols=3, sharex=True, figsize=(16,9))
        
        for i,ci in enumerate(patient_cis):
            cs12 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS1-2')]['Data'].values[0]
            cs34 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS3-4')]['Data'].values[0]
            cs56 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS5-6')]['Data'].values[0]
            
            cs12_ = np.convolve(abs(cs12), v, mode='valid')
            cs34_ = np.convolve(abs(cs34), v, mode='valid')
            cs56_ = np.convolve(abs(cs56), v, mode='valid')
            
            width_thresh=0.2
            
            if any(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_)):
                cs12_end_idx = np.argmax(cs12_) + np.argmax(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_))
            else:
                cs12_end_idx = len(cs12_)-1
            if any(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_)):  
                cs12_start_idx = np.argmax(cs12_) - np.argmax(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_))
            else:
                cs12_start_idx = 0
            if any(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_)):
                cs34_end_idx = np.argmax(cs34_) + np.argmax(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_))
            else:
                cs34_end_idx = len(cs34_)-1
            if any(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_)):  
                cs34_start_idx = np.argmax(cs34_) - np.argmax(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_))
            else:
                cs34_start_idx = 0
            if any(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_)):
                cs56_end_idx = np.argmax(cs56_) + np.argmax(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_))
            else:
                cs56_end_idx = len(cs56_)-1
            if any(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_)):  
                cs56_start_idx = np.argmax(cs56_) - np.argmax(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_))
            else:
                cs56_start_idx = 0
                        
            axes[i][0].plot(np.arange(len(cs12)), cs12/max(abs(cs12)), '--', np.arange(len(cs12_))+M/2, cs12_/max(abs(cs12_)))
            axes[i][0].axvline(x=np.argmax(cs12_)+M/2, color='k', linestyle='--')
            axes[i][0].axvline(x=cs12_start_idx + M/2, color='r', linestyle='--')
            axes[i][0].axvline(x=cs12_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][1].plot(np.arange(len(cs34)), cs34/max(abs(cs34)), '--', np.arange(len(cs34_))+M/2, cs34_/max(abs(cs34_)))
            axes[i][1].axvline(x=np.argmax(cs34_)+M/2, color='k', linestyle='--')
            axes[i][1].axvline(x=cs34_start_idx + M/2, color='r', linestyle='--')
            axes[i][1].axvline(x=cs34_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][2].plot(np.arange(len(cs56)), cs56/max(abs(cs56)), '--', np.arange(len(cs56_))+M/2, cs56_/max(abs(cs56_)))
            axes[i][2].axvline(x=np.argmax(cs56_)+M/2, color='k', linestyle='--')
            axes[i][2].axvline(x=cs56_start_idx + M/2, color='r', linestyle='--')
            axes[i][2].axvline(x=cs56_end_idx+M/2, color='r', linestyle='--')
            
        
        plt.suptitle('Convolved signal for: ' + patient_type + ' ' + patient)
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()
    

In [ ]:
%matplotlib qt
patient_types = X['Type'].unique()
M=14
v = np.ones(M)

for patient_type in patient_types:
    patients = X[X['Type']==patient_type]['Patient'].unique()
    if patient_type == 'avrt':
        patients = patients[1:]
        
    for patient in patients:
        patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='S1')]
        
        patient_cis = patient_X['Coupling Interval'].unique()
        patient_cis = np.sort(patient_cis)[::-1]

        fig, axes = plt.subplots(nrows=len(patient_cis), ncols=3, sharex=True, figsize=(16,9))
        
        for i,ci in enumerate(patient_cis):
            cs12 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS1-2')].iloc[0]['Data']
            cs34 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS3-4')].iloc[0]['Data']
            cs56 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS5-6')].iloc[0]['Data']
            
            cs12_ = np.convolve(abs(cs12), v, mode='valid')
            cs34_ = np.convolve(abs(cs34), v, mode='valid')
            cs56_ = np.convolve(abs(cs56), v, mode='valid')
            
            width_thresh=0.2
            
            if any(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_)):
                cs12_end_idx = np.argmax(cs12_) + np.argmax(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_))
            else:
                cs12_end_idx = len(cs12_)-1
            if any(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_)):  
                cs12_start_idx = np.argmax(cs12_) - np.argmax(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_))
            else:
                cs12_start_idx = 0
            if any(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_)):
                cs34_end_idx = np.argmax(cs34_) + np.argmax(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_))
            else:
                cs34_end_idx = len(cs34_)-1
            if any(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_)):  
                cs34_start_idx = np.argmax(cs34_) - np.argmax(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_))
            else:
                cs34_start_idx = 0
            if any(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_)):
                cs56_end_idx = np.argmax(cs56_) + np.argmax(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_))
            else:
                cs56_end_idx = len(cs56_)-1
            if any(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_)):  
                cs56_start_idx = np.argmax(cs56_) - np.argmax(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_))
            else:
                cs56_start_idx = 0
                        
            axes[i][0].plot(np.arange(len(cs12)), cs12/max(abs(cs12)), '--', np.arange(len(cs12_))+M/2, cs12_/max(abs(cs12_)))
            axes[i][0].axvline(x=np.argmax(cs12_)+M/2, color='k', linestyle='--')
            axes[i][0].axvline(x=cs12_start_idx + M/2, color='r', linestyle='--')
            axes[i][0].axvline(x=cs12_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][1].plot(np.arange(len(cs34)), cs34/max(abs(cs34)), '--', np.arange(len(cs34_))+M/2, cs34_/max(abs(cs34_)))
            axes[i][1].axvline(x=np.argmax(cs34_)+M/2, color='k', linestyle='--')
            axes[i][1].axvline(x=cs34_start_idx + M/2, color='r', linestyle='--')
            axes[i][1].axvline(x=cs34_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][2].plot(np.arange(len(cs56)), cs56/max(abs(cs56)), '--', np.arange(len(cs56_))+M/2, cs56_/max(abs(cs56_)))
            axes[i][2].axvline(x=np.argmax(cs56_)+M/2, color='k', linestyle='--')
            axes[i][2].axvline(x=cs56_start_idx + M/2, color='r', linestyle='--')
            axes[i][2].axvline(x=cs56_end_idx+M/2, color='r', linestyle='--')
            
        
        plt.suptitle('Convolved signal for: ' + patient_type + ' ' + patient)
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()
    

In [3]:
# Used to find S2 pulse using coupling_interval
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def get_segment(x, s, sr, coupling_interval=0, offset=10, plot_segment=True):
    # Get S1/S2 pulses from s
    s1s2_idxs = get_s1s2(s, sr, False)

    # Identify S2 pulse
    s1s2_intervals = np.diff(s1s2_idxs)
    s2_idx = find_nearest(s1s2_intervals, coupling_interval)
    s2_idx = s1s2_idxs[s2_idx+1]
    s1_idxs = np.asarray([i for i in s1s2_idxs if ((i>50) & (i<s2_idx))])
    # Add small offset
    s1_idxs += offset
    s2_idx += offset

    # For now just use strict segment length of 125.
    seg_length = 125
    s1_segs = []    # List of S1 segments
    s2_seg = x[s2_idx:(s2_idx+seg_length)] # S2 segment
    for s1_idx in s1_idxs:
        s1_segs.append(x[s1_idx:(s1_idx+seg_length)])


    if plot_segment:
        fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(16,4))
        ax1.plot(range(len(x)), x, range(s2_idx, s2_idx+seg_length), s2_seg, 'g')
        ax1.hold(True)
        for i, s1_seg in enumerate(s1_segs):
            ax1.plot(range(s1_idxs[i], (s1_idxs[i]+seg_length)), s1_seg, 'r')
        ax2.plot(range(len(s)), s)
        for idx in s1s2_idxs:
            ax2.axvline(idx, ymin=min(s), ymax=max(s), c='k', linestyle='--')
        plt.draw()
        plt.waitforbuttonpress()
        plt.close(fig)

    return np.array(s1_segs).astype(float), s2_seg

In [4]:
def epdata_main(file_loc, patients, patient_type, coupling_intervals):
    X_list = [] # Initially store data in a list of dictionaries
    column_names = ['Type', 'Patient', 'Coupling Interval', 'S1/S2', 'Data']    # Column names of pandas dataframe
    for patient in patients:
        for coupling_interval in coupling_intervals:
            # Extract data from file
            # file_name = file_loc + 'TESTEXPORT' + file + '.txt'
            try:
                file_name = file_loc + patient_type + patient + '/' + patient_type + 'patient' + patient + '-0' + coupling_interval + '.txt'
                data, sr, num_samples = parser.parseFile(file_name)
            except FileNotFoundError:
                try:
                    file_name = file_loc + patient_type + patient + '/' + patient_type + 'patient0' + patient + '-0' + coupling_interval + '.txt'
                    data, sr, num_samples = parser.parseFile(file_name)
                except:
                    print('Parsing error: coupling interval ' + coupling_interval + 'ms not available for patient ' + patient + '.')
                    continue

            # Use the following channels
            try:
                cs34 = data['CS 3-4'].values
                cs56 = data['CS 5-6'].values
            except KeyError:
                try:
                    cs34 = data['CS3-4'].values
                    cs56 = data['CS5-6'].values
                except KeyError:
                    print('Parsing error: could not extract CS3-4 and CS5-6 values for coupling interval ' + coupling_interval + 'ms.')
                    continue
            try:
                s = data['CSp'].values
                cs12 = data['CSd'].values
            except KeyError:
                try:
                    s = data['CS9-10'].values
                    cs12 = data['CS1-2'].values
                except KeyError:
                    try:
                        s = data['CS 9-10'].values
                        cs12 = data['CS 1-2'].values
                    except KeyError:
                        print('Parsing error: could not extract channel values for coupling interval ' + coupling_interval + 'ms.')
                        continue

            channel_names = ('CS1-2', 'CS3-4', 'CS5-6')
            channels = (cs12, cs34, cs56)
            offsets = (15, 12, 12)
            if  all(isinstance(i, np.int64) for i in s):
                for channel, channel_name, offset in zip(channels, channel_names, offsets):
                    if all(isinstance(i, np.int64) for i in channel):
                        # Get segment
                        s1_segs, s2_seg = get_segment(channel, s, sr, int(coupling_interval), offset, False)
                        temp_dict = {}
                        temp_dict.update({'Type': patient_type, 'Patient': patient, 'Coupling Interval': coupling_interval, 'Channel': channel_name, 'S1/S2': 'S2', 'Data': s2_seg})
                        X_list.append(temp_dict)
                        for i,s1_seg in enumerate(s1_segs):
                            temp_dict = {}
                            temp_dict.update({'Type': patient_type, 'Patient': patient, 'Coupling Interval': coupling_interval, 'Channel': channel_name, 'S1/S2': 'S1', 'Data': s1_segs[i]})
                            X_list.append(temp_dict)
                    else:
                        print('Parsing error: non-integer values in channel ' + channel_name + ' for coupling interval ' + coupling_interval + 'ms.')
            else:
                print('Parsing error: non-integer values in channels V1/CS9-10 for coupling interval ' + coupling_interval + 'ms.')
                continue

    X = pd.DataFrame(X_list)
    return X